In [1]:
#Making and Filling MDP
n = 4
states = [[0 for i in range(n*n)] for _ in range(2**(n*n))] # State is a list of the grid with 1 whenever the cell is lighted
gamma = 0.9

for i in range(2**(n*n)):
    x = i
    ind = (n*n)-1
    while x!=0:
        states[i][ind]=x%2
        ind-=1
        x=x//2

for i in range(2**(n*n)):
    states[i]=tuple(states[i])

LightsOutMdp = {
    tuple(i):{} for i in states
}

actions = [i for i in range(n*n)]
goal = states[0]

for state in states:
    transitions = {}
    if state == goal:
        for action in actions:
            transitions[action]=[(1, state, 0, True)]
        LightsOutMdp[tuple(state)]=transitions
        continue
    for action in actions:
        togs = [action]
        
        if (action+1)%n > action%n and action+1<n*n:
            togs.append(action+1)
        if (action-1)%n < action%n and action-1>=0:
            togs.append(action-1)
        if (action+n)<n*n:
            togs.append(action+n)
        if (action-n)>=0:
            togs.append(action-n)
        nex_st = list(state)
        for j in togs:
            nex_st[j]=(nex_st[j]+1)%2
        rew = 0
        if nex_st == goal:
            rew = 1
        transitions[action]=[(1, tuple(nex_st), rew, rew==1)] #(probability, next state, reward, is next state terminal)
    LightsOutMdp[tuple(state)]=transitions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

v = {} #ValueFunction
newv = {}
oldpolicy = {} #DeterministicPolicy
newpolicy = {}
for _ in states:
    v[tuple(_)]=10.0
    newv[tuple(_)] = 0.0
    oldpolicy[tuple(_)]=5
    newpolicy[tuple(_)] = 0
smol = 1e-9
goal = tuple(goal)
v[goal]=newv[goal]=1

while(newpolicy != oldpolicy):
    it+=1
    for _ in states:
        v[tuple(_)]=10.0
        newv[tuple(_)] = 0.0
    v[goal]=newv[goal]=1
    oldpolicy=newpolicy.copy()
    #PolicyEvaluation
    while(max(abs(newv[tuple(s)]-v[tuple(s)]) for s in states) > smol):
        v = newv.copy()
        for state in states:
            if state==goal:
                continue
            x = 0
            # print(it)
            l = LightsOutMdp[tuple(state)][newpolicy[tuple(state)]]
            for i in l:
                x+=i[0]*(i[2]+gamma*v[i[1]]*(i[3]==False))
            newv[tuple(state)]=x
    #PolicyImprovement
    for state in states:
        if state==goal:
            continue
        besta = 0
        bestexpre = -float('inf')
        for action in actions:
            expre = 0
            l = LightsOutMdp[tuple(state)][action]
            for i in l:
                expre+=i[0]*(i[2]+gamma*newv[i[1]]*(i[3]==False))
            if(expre>bestexpre):
                besta = action
                bestexpre = expre
        newpolicy[tuple(state)]=besta

print(it)


2
3
4
5
6
7
8
8
8


In [3]:
big = 1e2
m = 0
a1 = [tuple([1]*(n*n))]
for state in a1:
    if (state==goal):
        continue
    act = newpolicy[state]
    tup = LightsOutMdp[state][act][0]
    nexst = tup[1]
    movect = 1
    while(tup[3]==False):
        act = newpolicy[tuple(nexst)]
        tup = LightsOutMdp[tuple(nexst)][act][0]
        nexst = tup[1]
        movect+=1
        if (movect>big):
            movect=0
            break
    m=max(movect, m)
print(m)

5


In [19]:
#Playing the game
a1 = [tuple([0,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0])]
big = 1e2
movect=0
m=0

for state in a1:
    if (state==goal):
        print("DONE")
        continue
    act = newpolicy[state]
    tup = LightsOutMdp[state][act][0]
    nexst = tup[1]
    print('(', (act//4)+1, ', ', (act%4)+1, ')')
    for i in range(0,16, 4):
        for j in range(i, i+4):
            print(nexst[j], end=' ')
        print()
    movect = 1
    while(tup[3]==False and nexst!=goal):
        act = newpolicy[tuple(nexst)]
        tup = LightsOutMdp[tuple(nexst)][act][0]
        nexst = tup[1]

        print('(', (act//4)+1, ', ', (act%4)+1, ')')
        for i in range(0,16, 4):
            for j in range(i, i+4):
                print(nexst[j], end=' ')
            print()
        movect+=1
        if (movect>big):
            print("IMPOSSIBLE")
            movect=0
            break
    if(movect!=0):
        print("DONE")
    m=max(movect, m)
print(m)

( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 0 0 1 
0 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
1 1 0 1 
1 1 1 0 
0 1 0 1 
0 1 0 0 
( 1 ,  1 )
0 